In [1]:
!pip install gradientai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.3/166.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 20.9 MB/s eta 0:00:00


In [2]:
import os
os.environ['GRADIENT_ACCESS_TOKEN'] = "jNHWNUDxYWCDA5FL3eM9LWUkVf4nto2d"
os.environ['GRADIENT_WORKSPACE_ID'] = "fc9472b9-21be-4136-b210-5a9abdb5c169_workspace"

Read the files with new data set structure

In [7]:
import json

def convert (input_file):
  output_data = []
  with open(input_file, 'r') as file:
      for line in file:
          item = json.loads(line)
          user_message = item["messages"][1]["content"]
          assistant_message = item["messages"][2]["content"]
          formatted_data = {
              "inputs": f"### Instruction: {user_message}\n\n### Response: {assistant_message}"
          }
          output_data.append(formatted_data)
  return output_data

def saveFile(file_name,output_data):
  # Write the output data to a JSON file
  output_file = file_name

  with open(output_file, 'w') as file:
      json.dump(output_data, file, indent=4)

  print(f"Formatted data has been written to {output_file}")

In [13]:
# Specify the input JSONL file
trin_file = "/content/drive/MyDrive/train_data.jsonl"
valid_file = '/content/drive/MyDrive/validation_data.jsonl'
train_and_valid = []
out_file = 'train_Llama2.jsonl'
#convert train to new strcture
train = convert(trin_file)

valid = convert(valid_file)

train_and_valid = train + valid
saveFile(out_file,train_and_valid)


Formatted data has been written to train_Llama2.jsonl


In [21]:
# Specify the input JSONL file
test_file = "/content/drive/MyDrive/test_data.jsonl"

test = []
out_file = 'test_Llama2.jsonl'
#convert train to new strcture
test = convert(test_file)

saveFile(out_file,test)


Formatted data has been written to test_Llama2.jsonl


In [22]:
print(len(train))
print(len(valid))
print(len(train) + len(valid))
print(len(train_and_valid))
print(len(test))

178
23
201
201
44


Trian Model

In [12]:
import json
#read training data
file_path = '/content/drive/MyDrive/train_Up_Llama2.jsonl'  # Replace with the actual path to your JSON file

with open(file_path, 'r') as file:
    # Load the JSON data
    data_train = json.load(file)
print(len(data_train))

201


In [13]:
from gradientai import Gradient

def main():

  with Gradient() as gradient:
      #nous-hermes2 is a fine tuned vesion of Llama-2 13B Nous Hermes 2
      base_model = gradient.get_base_model(base_model_slug="nous-hermes2")

      #make a copy of the base model aims to fine-tuned
      new_model_adapter = base_model.create_model_adapter(
          name="CCSIT_InfoBot"
      )

      # this is where fine-tuning happens
      # num_epochs is the number of times you fine-tune the model
      # more epochs tends to get better results, but you also run the risk of "overfitting"
      # play around with this number to find what works best for you
      num_epochs = 3
      batch_size_limit = 100
      count = 0

      while count < num_epochs:
          print(f"Fine-tuning the model, iteration {count + 1}")

          # Ensure that each iteration uses a batch size within the limit
          for start_idx in range(0, len(data_train), batch_size_limit):
              end_idx = start_idx + batch_size_limit
              batch_data = data_train[start_idx:end_idx]

              # Call the fine_tune method with the current batch
              new_model_adapter.fine_tune(samples=batch_data)

          count += 1


if __name__ == "__main__":
    main()

Fine-tuning the model, iteration 1
Fine-tuning the model, iteration 2
Fine-tuning the model, iteration 3


In [16]:
from gradientai import Gradient

def main():
  with Gradient() as gradient:

      #nous-hermes2 is a fine tuned vesion of Llama-2 13B Nous Hermes 2
      base_model = gradient.get_base_model(base_model_slug="nous-hermes2")

      #make a copy of the base model aims to fine-tuned
      new_model_adapter = base_model.create_model_adapter(
          name="CCSIT_InfoBot"
      )
      sample_query = "### Instruction: What is the student population of the IAU university?\n\n### Response:"
      # after fine-tuning
      completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
      print(f"Generated (after fine-tune): {completion}")

#new_model_adapter.delete()
if __name__ == "__main__":
    main()


Generated (after fine-tune):  I'm sorry, but I don't have information about the student population of the IAU university. Could you please provide more context or clarify your question?


In [ ]:
import json
import pandas as pd
file_path = '/content/drive/MyDrive/test_Up_Llama2.jsonl'  # Replace with the actual path to your JSON file

with open(file_path, 'r') as file:
    # Load the JSON data
    data_train = json.load(file)
final_df = pd.DataFrame(columns=['instruction', 'response','Generate Response'])
for i in range(len(data_train)):
  l = data_train[i]
  instruction = l['inputs'].split('### Response:')[0].strip() + '\\n\\n ### Response:'
  response = '### Response:' + l['inputs'].split('### Response:')[1].strip()
  temp_df = pd.DataFrame({
    'instruction': [instruction],
    'response': [response],
    'Generate Response' :''
  })

  # Append the temporary DataFrame to the final DataFrame
  final_df = final_df.append(temp_df, ignore_index=True)

In [22]:

from gradientai import Gradient

def main():
  with Gradient() as gradient:

      #nous-hermes2 is a fine tuned vesion of Llama-2 13B Nous Hermes 2
      base_model = gradient.get_base_model(base_model_slug="nous-hermes2")

      #make a copy of the base model aims to fine-tuned
      new_model_adapter = base_model.create_model_adapter(
          name="CCSIT_InfoBot"
      )
      for i in range(len(final_df)):
        sample_query = final_df['instruction'][i]
        # after fine-tuning
        completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
        print(f"Generated (after fine-tune): {completion}")
        final_df['Generate Response'][i] = completion

#new_model_adapter.delete()
if __name__ == "__main__":
    main()

Generated (after fine-tune):  I'm sorry but I don't have any information about the IAU university. Could you please provide more details or context about the university you are referring to?
Generated (after fine-tune):  The College of Computer Science and Information Technology was established in 2001.
Generated (after fine-tune):  The goals of the College of Engineering at the University of California, Berkeley are to:
1. Provide a world-class engineering education that prepares students for successful careers in industry, academia, and government.
2. Conduct cutting-edge research that advances the field of engineering and has a positive impact on society.
3. Foster an inclusive and diverse community that values and promotes collaboration, innovation, and creativity.
4. Engage
Generated (after fine-tune):  The College's approach to financial sustainability involves a combination of strategic planning, cost management, and revenue generation initiatives. The College's leadership team 

In [23]:
final_df.head()

,instruction,response,Generate Response
0,### Instruction: What is the student populatio...,### Response:The student population of the uni...,I'm sorry but I don't have any information ab...
1,### Instruction: When was the College of Compu...,### Response:The College was established in 2010.,The College of Computer Science and Informati...
2,### Instruction: What are some of the goals of...,### Response:Goals include offering quality ed...,The goals of the College of Engineering at th...
3,### Instruction: What is the College's approac...,### Response:The College aims to strengthen fi...,The College's approach to financial sustainab...
4,### Instruction: Can you share some examples o...,### Response:Research areas at CCSIT include a...,Certainly! Here are some examples of research...


In [24]:
file_path = 'first_trail_Llama2.csv'
final_df.to_csv(file_path, index=False)